In [2]:
## import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv

--2025-11-01 12:14:28--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874188 (854K) [text/plain]
Saving to: ‘car_fuel_efficiency.csv.1’

car_fuel_efficiency 100%[===================>] 853.70K  --.-KB/s    in 0.006s  

2025-11-01 12:14:29 (147 MB/s) - ‘car_fuel_efficiency.csv.1’ saved [874188/874188]



In [4]:
df = pd.read_csv("car_fuel_efficiency.csv")

In [5]:
print(df.info())
print(df.shape)
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9704 entries, 0 to 9703
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   engine_displacement  9704 non-null   int64  
 1   num_cylinders        9222 non-null   float64
 2   horsepower           8996 non-null   float64
 3   vehicle_weight       9704 non-null   float64
 4   acceleration         8774 non-null   float64
 5   model_year           9704 non-null   int64  
 6   origin               9704 non-null   object 
 7   fuel_type            9704 non-null   object 
 8   drivetrain           9704 non-null   object 
 9   num_doors            9202 non-null   float64
 10  fuel_efficiency_mpg  9704 non-null   float64
dtypes: float64(6), int64(2), object(3)
memory usage: 834.1+ KB
None
(9704, 11)
       engine_displacement  num_cylinders   horsepower  vehicle_weight  \
count          9704.000000    9222.000000  8996.000000     9704.000000   
mean           

In [6]:
print(df.select_dtypes(include="object").value_counts())

origin  fuel_type  drivetrain       
Asia    Gasoline   All-wheel drive      877
Europe  Gasoline   All-wheel drive      825
Asia    Diesel     Front-wheel drive    822
Europe  Diesel     All-wheel drive      821
                   Front-wheel drive    811
USA     Gasoline   Front-wheel drive    805
                   All-wheel drive      805
        Diesel     Front-wheel drive    804
Europe  Gasoline   Front-wheel drive    797
Asia    Gasoline   Front-wheel drive    789
USA     Diesel     All-wheel drive      789
Asia    Diesel     All-wheel drive      759
Name: count, dtype: int64


In [7]:
### data quality checks 
def check_outliers_zscore(series, threshold=3):
    # Calculate Z-scores
    z_scores = np.abs((series - series.mean()) / series.std())
    
    # Identify outliers
    outliers = series[z_scores > threshold]
    
    return {
        'count': len(outliers),
        'percentage': len(outliers) / len(series) * 100
    }

def validate_dataframe_comprehensive(df):
    """
    Performs a comprehensive data quality check on the input DataFrame.
    """
    # 1. Initial Format Check 
    if not isinstance(df, pd.DataFrame):
        return False, "Invalid DataFrame format. Expected a pandas DataFrame."
    dtype_map = {
        "engine_displacement": "Int64",
        "num_cylinders": "float64",
        "horsepower": "float64",
        "vehicle_weight": "float64",
        "acceleration": "float64",
        "model_year": "Int64",
        "origin": "object",
        "fuel_type": "object",
        "drivetrain": "object",
        "num_doors": "float64",
        "fuel_efficiency_mpg":"float64",
    }        
    # Check for missing/extra columns
    if set(df.columns) != set(dtype_map.keys()):
        print(f"Column mismatch: Missing: {set(dtype_map.keys()) - set(df.columns)}")
        print(f"Column mismatch: Extra: {set(df.columns) - set(dtype_map.keys())}")
        return False, "Schema structure is incorrect."
        
    # check the data structure of the attributes
    for col, expected_dtype in dtype_map.items():
        if str(df[col].dtype) != expected_dtype.lower():
            return False, f"Invalid data type for {col}. Expected: {expected_dtype}."

    # check for any null values and duplicate values
    for col in dtype_map:
        null_counts = df[col].isna().sum()
        
        if null_counts > 0:
            print(f"{col} has {null_counts} null values.")
    dup_counts = df.duplicated().sum()
    print(f"DataFrame df has {dup_counts} duplicate rows.")
    
    # how to check for numerical outliers
    for col in df.select_dtypes(include='number'):
        report = check_outliers_zscore(df[col])
        if report['count'] > 0:
            print(f"Outliers for {col}: {report['count']} ({report['percentage']:.2f}%)")
        else:
            continue
    return True, "DataFrame is valid."

In [8]:
### Preparing the dataset
validate_dataframe_comprehensive(df)

num_cylinders has 482 null values.
horsepower has 708 null values.
acceleration has 930 null values.
num_doors has 502 null values.
DataFrame df has 0 duplicate rows.
Outliers for engine_displacement: 39 (0.40%)
Outliers for num_cylinders: 81 (0.83%)
Outliers for horsepower: 19 (0.20%)
Outliers for vehicle_weight: 28 (0.29%)
Outliers for acceleration: 25 (0.26%)
Outliers for num_doors: 5 (0.05%)
Outliers for fuel_efficiency_mpg: 27 (0.28%)


(True, 'DataFrame is valid.')

In [9]:
# fill missing values with zeros
df1 = df.fillna(0)
print(df1.isna().any())

engine_displacement    False
num_cylinders          False
horsepower             False
vehicle_weight         False
acceleration           False
model_year             False
origin                 False
fuel_type              False
drivetrain             False
num_doors              False
fuel_efficiency_mpg    False
dtype: bool


## Question 1

Let's train a decision tree regressor to predict the `fuel_efficiency_mpg` variable. 

* Train a model with `max_depth=1`.


Which feature is used for splitting the data?


* `'vehicle_weight'`
* `'model_year'`
* `'origin'`
* `'fuel_type'`


In [10]:
### Question 1
# Do train/validation/test split with 60%/20%/20% distribution. 
# Use the `train_test_split` function and set the `random_state` parameter to 1.
# Use `DictVectorizer(sparse=True)` to turn the dataframes into matrices.

df_train, df_temp = train_test_split(df1, 
                              random_state=1, 
                              test_size=0.4)
df_val, df_test = train_test_split(df_temp, 
                                   random_state=1,
                                   test_size=0.5)
target = "fuel_efficiency_mpg"
train_y = df_train[target]
df_train_X = df_train.drop(columns=[target])
test_y = df_test[target]
df_test_X = df_test.drop(columns=[target])
val_y = df_val[target]
df_val_X = df_val.drop(columns=[target])

# vectorize into matrix
v = DictVectorizer(sparse=True)
train_X_matrix = v.fit_transform(df_train_X.to_dict(orient='records'))
val_X_matrix = v.transform(df_val_X.to_dict(orient='records'))
test_X_matrix = v.transform(df_test_X.to_dict(orient='records'))

# train a decision tree regressor to predict the `fuel_efficiency_mpg` variable
DT = DecisionTreeRegressor(max_depth=4, random_state=42)
DT.fit(train_X_matrix, train_y)

# print feature importance
importances = DT.feature_importances_

# Get the index of the most important feature
idx = np.argmax([importances])

# Get the feature name with the highest feature importance
feature_names = v.feature_names_
print(feature_names[idx])

vehicle_weight



## Question 2

Train a random forest regressor with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1` (optional - to make training faster)

$$
\text{MSE=} \sum^n_{i=1}{(y_i - \hat{y_i})^2}
$$
$$
\text{RMSE=} \sqrt{\frac{\text{MSE}}{n}}
$$
What's the RMSE of this model on the validation data?

* 0.045
* 0.45
* 4.5
* 45.0

In [15]:
### Question 2
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=10, random_state=1,n_jobs=-1)
rf.fit(train_X_matrix, train_y)

pred_y = rf.predict(val_X_matrix)

## compute R^2
u = ((val_y - pred_y)** 2).sum()
v = ((val_y - val_y.mean()) ** 2).sum()
n = len(val_y)
R_squared = 1-(u/v)
RMSE = np.sqrt(u/n)

print(f"R square is {R_squared}")
print(f"RMSE is {RMSE}")

R square is 0.9673841548317689
RMSE is 0.4602815367032659



## Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10.
* Set `random_state` to `1`.
* Evaluate the model on the validation dataset.


After which value of `n_estimators` does RMSE stop improving?
Consider 3 decimal places for calculating the answer.

- 10
- 25
- 80
- 200

If it doesn't stop improving, use the latest iteration number in
your answer.


In [24]:
### Question 3
import numpy as np
from sklearn.ensemble import RandomForestRegressor

min_rmse = float('inf')  # Tracks the best (lowest) RMSE found across all iterations.
best_n_estimator = 0     # Tracks the n_estimator corresponding to the min_rmse.
results = {}             # Stores all results for later inspection (n: rmse)


for n in range(10, 201, 10):
    # 1. Fit with Random Forest Regressor
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1,warm_start=True)
    rf.fit(train_X_matrix, train_y)

    pred_y = rf.predict(val_X_matrix)
    
    # 2. Compute RMSE
    u = ((val_y - pred_y)** 2).sum()
    current_rmse = np.sqrt(u / len(val_y))

    # 3. Store the result
    results[n] = current_rmse
    
    # 4. Compare the current RMSE to the overall 
    if current_rmse < min_rmse:
        min_rmse = current_rmse      # Update the overall minimum RMSE
        best_n_estimator = n         # Update the n_estimator that achieved it
        
print(f"All RMSE results: {results}")
print("#" * 20)
print(f"The best n_estimator is: {best_n_estimator}")
print(f"The minimum RMSE achieved is: {min_rmse:.4f}")

All RMSE results: {10: np.float64(0.4602815367032658), 20: np.float64(0.44615674589110027), 30: np.float64(0.4397780761280069), 40: np.float64(0.4383939265191818), 50: np.float64(0.43717032494674524), 60: np.float64(0.4355914081920472), 70: np.float64(0.43611238591302576), 80: np.float64(0.43605455887808786), 90: np.float64(0.43541008234407647), 100: np.float64(0.4352773655478666), 110: np.float64(0.434896815770466), 120: np.float64(0.43546652508605704), 130: np.float64(0.4349233620666645), 140: np.float64(0.4351068229164201), 150: np.float64(0.4351910645153306), 160: np.float64(0.4352369042756664), 170: np.float64(0.43520773900215154), 180: np.float64(0.4352404093499596), 190: np.float64(0.43539799338117574), 200: np.float64(0.4350031248889441)}
####################
The best n_estimator is: 110
The minimum RMSE achieved is: 0.4349


## Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values,
  * try different values of `n_estimators` from 10 till 200 (with step 10)
  * calculate the mean RMSE 
* Fix the random seed: `random_state=1`


What's the best `max_depth`, using the mean RMSE?

* 10
* 15
* 20
* 25

In [29]:
### Question 4 to find the best max_depth, using the mean RMSE
from sklearn.model_selection import GridSearchCV

base_rf = RandomForestRegressor(random_state=1, n_jobs=-1)
param_grid = {
    'max_depth': [20, 25, 10, 15],
    'n_estimators': list(range(10,201,10))
}

min_mean_rmse = float('inf') # Tracks the overall lowest mean RMSE
best_max_depth = 0
mean_rmse_results = {}

for m in param_grid['max_depth']:
    # initialize the model using the current max_depth
    rf = RandomForestRegressor(
        max_depth=m, 
        random_state=1, 
        n_jobs=-1,
        warm_start=True 
    )
    # set an array to aggregate the scores
    rmse_scores = []
    
    for n in param_grid['n_estimators']:
        rf.set_params(n_estimators=n)
        rf.fit(train_X_matrix, train_y)
        pred_y = rf.predict(val_X_matrix)
        
        # Compute RSME
        u = ((val_y - pred_y)**2).sum()
        RMSE = np.sqrt(u / len(val_y))

        # Store the individual RMSE score for this depth
        rmse_scores.append(RMSE)
        
    mean_rmse = np.mean(rmse_scores)
    mean_rmse_results[m] = mean_rmse 

    # Compare the current RMSE to the overall 
    if mean_rmse < min_mean_rmse:
        min_mean_rmse = mean_rmse      
        best_max_depth = m        
print(f"Best max_depth: {best_max_depth}")
    

Best max_depth: 10



# Question 5

We can extract feature importance information from tree-based models. 

At each step of the decision tree learning algorithm, it finds the best split. 
When doing it, we can calculate "gain" - the reduction in impurity before and after the split. 
This gain is quite useful in understanding what are the important features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the
[`feature_importances_`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor.feature_importances_)
field. 

For this homework question, we'll find the most important feature:

* Train the model with these parameters:
  * `n_estimators=10`,
  * `max_depth=20`,
  * `random_state=1`,
  * `n_jobs=-1` (optional)
* Get the feature importance information from this model


What's the most important feature (among these 4)? 

* `vehicle_weight`
*	`horsepower`
* `acceleration`
* `engine_displacement`	



In [36]:
v = DictVectorizer(sparse=True)
v.fit_transform(df_train_X.to_dict(orient='records'))
print(v.feature_names_)

['acceleration', 'drivetrain=All-wheel drive', 'drivetrain=Front-wheel drive', 'engine_displacement', 'fuel_type=Diesel', 'fuel_type=Gasoline', 'horsepower', 'model_year', 'num_cylinders', 'num_doors', 'origin=Asia', 'origin=Europe', 'origin=USA', 'vehicle_weight']


In [37]:
## Question 5
rf = RandomForestRegressor(n_estimators=10, max_depth=20,random_state=1,n_jobs=-1)
rf.fit(train_X_matrix,train_y)

# record feature importance
importances = rf.feature_importances_
idx = np.argmax([importances])

# find the most important feature
feature_names = v.feature_names_
print(feature_names[idx])

vehicle_weight



## Question 6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter:

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

```
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
```

Now change `eta` from `0.3` to `0.1`.

Which eta leads to the best RMSE score on the validation dataset?

* 0.3
* 0.1
* Both give equal value



In [40]:
#!pip install xgboost

In [45]:
import xgboost as xgb
from xgboost import XGBRegressor

# 1. Set parameters dictionary
xgb_params = {
    "eta": 0.1,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "nthread": 4,
    "seed": 42
}

# 2. Initialize the model 
# Pass the parameters and set the number of estimators (trees)
model = XGBRegressor(
    n_estimators=100, # Corresponds to num_boost_round
    objective='reg:squarederror', # Explicitly set for standard regression
    **xgb_params # Unpack the remaining parameters
)

# 3. Fit the model 
# Use your original data matrices (assuming val_X and val_y are NumPy arrays/Pandas Series)
model.fit(
    train_X_matrix, train_y,
    # Use the validation set for early stopping if desired
    eval_set=[(val_X_matrix, val_y)],
    verbose=False
)

# 4. Make predictions 
pred_y = model.predict(val_X_matrix)

# 5. Evaluate performance RMSE
u = ((val_y - pred_y)**2).sum()
RMSE = np.sqrt(u / len(val_y))

print(f"RMSE is {RMSE}")

RMSE is 0.42799904532270117


In [46]:
import xgboost as xgb
from xgboost import XGBRegressor

# 1. Set parameters dictionary to eta = 0.3
xgb_params = {
    "eta": 0.3,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "nthread": 4,
    "seed": 42
}

# 2. Initialize the model 
# Pass the parameters and set the number of estimators (trees)
model = XGBRegressor(
    n_estimators=100, # Corresponds to num_boost_round
    objective='reg:squarederror', # Explicitly set for standard regression
    **xgb_params # Unpack the remaining parameters
)

# 3. Fit the model 
# Use your original data matrices (assuming val_X and val_y are NumPy arrays/Pandas Series)
model.fit(
    train_X_matrix, train_y,
    # Use the validation set for early stopping if desired
    eval_set=[(val_X_matrix, val_y)],
    verbose=False
)

# 4. Make predictions 
pred_y = model.predict(val_X_matrix)

# 5. Evaluate performance RMSE
u = ((val_y - pred_y)**2).sum()
RMSE = np.sqrt(u / len(val_y))

print(f"RMSE is {RMSE}")

RMSE is 0.4595951501538645


$\text{eta} = 0.1$: This is a lower learning rate. The model takes smaller, more cautious steps. This typically requires a larger number of boosting rounds (num_boost_round or n_estimators) but often results in a more robust model that generalizes better to unseen data.

## Submit the results

* Submit your results here: https://courses.datatalks.club/ml-zoomcamp-2025/homework/hw06
* If your answer doesn't match options exactly, select the closest one. If the answer is exactly in between two options, select the higher value.
